In [1]:
import requests as r
import pandas as pd
import shutil
import os
import chardet
from zipfile import ZipFile

dev = False

tmp_dir = 'julienne_data/'
start = 2012
end = 2018
ipeds_url = 'https://nces.ed.gov/ipeds/datacenter/data/'
url_pairs = [#('HD', ''),
#              ('IC', ''),
            ('IC', '_AY')#,
  #           ('EF', 'A_DIST'),
   #          ('SAL', '_IS'),
    #         ('OM', '')
            ]
ipeds_cols = {
    'HD'       : ['UNITID', 'INSTNM', 'IALIAS', 'ADDR', 'CITY', 'STABBR', 'ZIP', 'FIPS', 'OBEREG', 'OPEFLAG', 'WEBADDR', 'SECTOR', 'ICLEVEL', 'CONTROL', 'HLOFFER', 'UGOFFER', 'GROFFER', 'HDEGOFR1', 'DEGGRANT', 'HBCU', 'OPENPUBL', 'ACT', 'NEWID', 'DEATHYR', 'CYACTIVE', 'PSET4FLG', 'INSTCAT', 'C15BASIC', 'C15IPUG', 'C15IPGRD', 'C15UGPRF', 'C15ENPRF', 'C15SZSET', 'CCBASIC', 'CARNEGIE', 'LANDGRNT', 'INSTSIZE', 'F1SYSTYP', 'F1SYSNAM', 'F1SYSCOD', 'COUNTYCD', 'COUNTYNM', 'CNGDSTCD', 'LONGITUD', 'LATITUDE'],
    'IC'       : ['UNITID', 'CNTLAFFI', 'PUBPRIME', 'PUBSECON', 'RELAFFIL', 'DSTNUGC', 'DSTNUGP', 'DSTNUGN', 'DSTNGC', 'DSTNGP', 'DSTNGN', 'DISTCRS', 'DISTPGS', 'DSTNCED1', 'DSTNCED2', 'DSTNCED3', 'DISTNCED', 'ALLONCAM'],
    'IC_AY'    : ['UNITID', 'TUITION1', 'FEE1', 'HRCHG1', 'TUITION2', 'FEE2', 'HRCHG2', 'TUITION3', 'FEE3', 'HRCHG3', 'TUITION5', 'FEE5', 'HRCHG5', 'TUITION6', 'FEE6', 'HRCHG6', 'TUITION7', 'FEE7', 'HRCHG7'],
    'EFA_DIST' : ['UNITID', 'EFDELEV', 'EFDETOT', 'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1', 'EFDEEX2', 'EFDEEX3', 'EFDEEX4', 'EFDEEX5'],
    'SAL_IS'   : ['UNITID', 'ARANK', 'SAINSTT', 'SAINSTM', 'SAINSTW', 'SA_9MCT', 'SA_9MCM', 'SA_9MCW', 'SATOTLT', 'SATOTLM', 'SATOTLW', 'SAOUTLT', 'SAOUTLM', 'SAOUTLW', 'SAEQ9OT', 'SAEQ9OM', 'SAEQ9OW', 'SAEQ9AT', 'SAEQ9AM', 'SAEQ9AW'],
    'OM'       : ['UNITID', 'OMCHRT', 'OMACHRT', 'OMAWDP8', 'OMENRTP', 'OMENRYP', 'OMENRAP', 'OMENRUP']
}

if os.path.isdir(tmp_dir) and not dev:
    shutil.rmtree(tmp_dir)

if not (dev and os.path.isdir(tmp_dir)):
    os.mkdir(tmp_dir)
    


In [2]:
# Dataframes to be populated and merged per year: hd_df, ic_df, icay_df, efdist_df, salis_df, om_df

for year in range(start, end):
    df = {}
    for prefix, suffix in url_pairs:
        
        # Skip missing datasets
        if prefix == "OM" and year < 2015:
            continue
            
        # Download and process the existing data
        else:
            url = ipeds_url + prefix + str(year) + suffix + '.zip'
            zip = tmp_dir + prefix + str(year) + suffix + '.zip'
            file = prefix.lower() + str(year) + suffix.lower() + '.csv'
            stream = r.get(url, stream=True)    
            print(str(stream.status_code) + '\t' + url)
            if stream.status_code == 200:
                with open(zip, 'wb') as out_file:
                    shutil.copyfileobj(stream.raw, out_file)
                z = ZipFile(zip).open(file)
                ZipFile(zip).extract(file, tmp_dir)
                enc_result = chardet.detect(z.read())
                tmp_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])
            else:
                print('Failed to retrieve: ' + url)

        # Eliminate and calculate columns here        
        # Set null values for missing but expected columns
        for col in ipeds_cols[prefix + suffix]:
            if col not in tmp_df.columns:
                tmp_df[col] = 'Null'
        # First the easy ones: HD, IC, IC_AY - just take the columns we need
        if (prefix, suffix) in [('HD', ''), ('IC', ''), ('IC', '_AY')]:
            df[prefix + suffix] = tmp_df[ipeds_cols[prefix + suffix]]
        elif (prefix, suffix) == ('EF', 'A_DIST'):
            # Transpose rows to cols on EFDELEV (drop and recalculate)
            continue
        elif (prefix, suffix) == ('SAL', '_IS'):
            # Transpose rows to cols on EFDELEV
            continue
        elif (prefix, suffix) == ('OM', ''):
            # Transpose rows to cols on EFDELEV
            continue
        else:
            print("Don't know what to do with this file: " + file)
            quit()
    
    # Append columns into working year (add year)
    
    
    # And initiate or merge to master frame
#     if year == start:
#         ipeds = __MERGED_DF__
#         ipeds['YEAR'] = year
#     else
#         pd.concat(ipeds, merged, ignore_index=True)
    
# Recode categorical values
        
        

200	https://nces.ed.gov/ipeds/datacenter/data/IC2012_AY.zip
200	https://nces.ed.gov/ipeds/datacenter/data/IC2013_AY.zip
200	https://nces.ed.gov/ipeds/datacenter/data/IC2014_AY.zip
200	https://nces.ed.gov/ipeds/datacenter/data/IC2015_AY.zip
200	https://nces.ed.gov/ipeds/datacenter/data/IC2016_AY.zip
200	https://nces.ed.gov/ipeds/datacenter/data/IC2017_AY.zip


In [3]:
# tmp_df
df[prefix + suffix]
# new_df = tmp_df[['UNITID', 'XOMRCHRT']]
# new_df

,UNITID,TUITION1,FEE1,HRCHG1,TUITION2,FEE2,HRCHG2,TUITION3,FEE3,HRCHG3,TUITION5,FEE5,HRCHG5,TUITION6,FEE6,HRCHG6,TUITION7,FEE7,HRCHG7
0,100654,8379,1478,274,8379,1478,274,16758,1478,548,8379,1478,418,8379,1478,418,16758,1478,836
1,100663,8328,0,347,8328,0,347,19032,0,793,7866,0,437,7866,0,437,17910,0,995
2,100690,9000,900,430,9000,900,430,9000,900,430,11700,1300,750,11700,1300,750,11700,1300,750
3,100706,9356,924,411,9356,924,411,20556,924,908,10224,546,636,10224,546,636,22696,546,1402
4,100724,8328,2740,347,8328,2740,347,16656,2740,694,7408,2740,412,7408,2740,412,14832,2740,824
5,100751,10780,0,586,10780,0,586,28100,0,1336,10780,0,640,10780,0,640,28100,0,1470
6,100760,3570,870,119,3570,870,119,7140,870,238,.,.,.,.,.,.,.,.,.
7,100812,5940,750,198,5940,750,198,11880,750,396,7800,0,325,7800,0,325,7800,0,325
8,100830,7320,700,305,7320,700,305,16440,700,685,6930,640,385,6930,640,385,15588,640,866
9,100858,9336,1632,389,9336,1632,389,28008,1632,1167,9342,1632,519,9342,1632,519,28026,1632,1557


In [4]:
# # Dataframes to be populated and merged per year: hd_df, ic_df, icay_df, efdist_df, salis_df, om_df

# base_url = 'https://nces.ed.gov/ipeds/datacenter/data/'

# for year in range(start, end):
#     # HD - Institutional Characteristics - Directory information
#     url = base_url + 'HD' + str(year) + '.zip'
#     zip = tmp_dir + 'HD' + str(year) + '.zip'
#     file = 'hd' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # IC - Institutional Characteristics - Educational offerings, organization, services and athletic associations
#     url = base_url + 'IC' + str(year) + '.zip'
#     zip = tmp_dir + 'IC' + str(year) + '.zip'
#     file = 'ic' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         ic_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # ICAY - Institutional Characteristics	Student charges for academic year programs
#     url = base_url + 'IC' + str(year) + '_AY.zip'
#     zip = tmp_dir + 'IC' + str(year) + 'AY.zip'
#     file = 'ic' + str(year) + '_ay.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # EFDIST - Fall Enrollment - Distance education status and level of student: Fall 2017
#     url = base_url + 'EF' + str(year) + 'A_DIST.zip'
#     zip = tmp_dir + 'EF' + str(year) + 'A_DIST.zip'
#     file = 'ef' + str(year) + 'a_dist.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # SAL_IS - Instructional Staff/Salaries - Number and salary outlays for full-time nonmedical instructional staff, by gender, and academic rank
#     url = base_url + 'SAL' + str(year) + '_IS.zip'
#     zip = tmp_dir + 'SAL' + str(year) + '_IS.zip'
#     file = 'sal' + str(year) + '_is.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])

#     # OM - Outcome Measures - Award and enrollment data at four, six and eight years of entering degree/certificate-seeking undergraduate cohorts
#     url = base_url + 'OM' + str(year) + '.zip'
#     zip = tmp_dir + 'OM' + str(year) + '.zip'
#     file = 'om' + str(year) + '.csv'
#     stream = r.get(url, stream=True)
#     if stream.status_code == 200:
#         with open(zip, 'wb') as out_file:
#             shutil.copyfileobj(stream.raw, out_file)
#         z = ZipFile(zip).open(file)
#         ZipFile(zip).extract(file, tmp_dir)
#         enc_result = chardet.detect(z.read())
#         hd_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])
        
        

In [5]:
if not dev:
    shutil.rmtree(tmp_dir)